In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import unidecode
import os

In [119]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

#### Funciones

In [296]:
def doClean(a):   
    a = str(a)
    # Quitar las expresiones tras la barra /
    a = re.sub('\/.*', '', a)
    # Quitar las aclaraciones entre paréntesis
    a = re.sub('\(.*\)', '', a)
    # Quitar acentos 
    a = unidecode.unidecode(a)
    # Quitar aquello que no sean palabras o cosa que se le parezca
    a = re.sub('\W', ' ', a) 
    # Quitar espacios extra en caso de haber
    a = re.sub('\s+', ' ', a)
    # Pasar el texto a minúsulas
    a = a.lower() 
    a = a.lstrip()
    # Quitar espacio derecha
    a = a.rstrip()
    a = re.sub(' ', '_', a)
    #a = a.strip() 
    
    return a

In [640]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=1):
    """
    df_1 is the left table to join
    df_2 is the right table to join
    key1 is the key column of the left table
    key2 is the key column of the right table
    threshold is how close the matches should be to return a match, based on Levenshtein distance
    limit is the amount of matches that will get returned, these are sorted high to low
    """
    s = df_2[key2].tolist()

    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m

    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2

    return df_1

#### Carga y union de ficheros

In [164]:
os.getcwd()

'/home/patcalsi/projects/Exodus_urbano/data/patri'

In [165]:
os.listdir()

['sociedad_seguridad_guardia_civil_2019_limpio.csv',
 'union_ficheros.ipynb',
 'sociedad_educacion_universidad_2018_limpio.csv',
 'sociedad_sanidad_hospitales_2019_limpio.csv',
 'union_patri.csv',
 'sociedad_educacion_colegios_2020_limpio.csv',
 'sociedad_sanidad_centros_salud_2019_limpio.csv',
 '.ipynb_checkpoints',
 '.~lock.Empresas-SICTED2.xls#',
 'economia_turismo_establecimientos_20XX_limpio.csv',
 'tratamiento',
 'sociedad_usos_suelo_2019_limpio.csv',
 'manual.txt',
 'sociedad_cultura_bibliotecas_2017_limipio.csv',
 'listado-codigos-postales-con-LatyLon.csv',
 'sociedad_educacion_nivel_formacion_ccaa_2018.csv']

## Fichero maestro

In [1070]:
maestro_municipios = pd.read_csv("../maestro_idiomas_ine.csv", index_col=0, dtype={'postal_code': object, 'ine_code':object})
maestro_municipios.shape

(14729, 17)

In [1071]:
#correcciones
maestro_municipios['province'].replace("islas_baleares", "illes_balears", inplace=True )
maestro_municipios['cities_es/province'].replace("islas_baleares", "illes_balears", inplace=True, regex=True)
maestro_municipios['ccaa'].replace("islas_baleares", "illes_balears", inplace=True, regex=True)
maestro_municipios['ccaa'].replace("comunidad_valenciana", "comunitat_valenciana", inplace=True, regex=True)
maestro_municipios['province'].replace("guipuzcoa", "gipuzkoa", inplace=True )
#maestro_municipios['cities_es/province'].replace("guipuzcoa", "gipuzkoa", inplace=True, regex=True)

#eliminacion nans
maestro_municipios = maestro_municipios[~maestro_municipios['province'].isnull()]

In [1072]:
maestro_municipios.head()

,ine_code,postal_code,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province_es,cities_es/province,cities_es/ccaa_es,cities_es/ccaa,cities/province_es,province_es/ccaa_es,province_es/ccaa,province/ccaa_es,province/ccaa
0,01001,01240,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/alava,alegria_dulantzi/araba,alegria_dulantzi/pais_vasco,alegria_dulantzi/pais_vasco,alegria_dulantzi/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco
1,01001,01193,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/alava,alegria_dulantzi/araba,alegria_dulantzi/pais_vasco,alegria_dulantzi/pais_vasco,alegria_dulantzi/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco
2,01002,01450,amurrio,amurrio,alava,araba,pais_vasco,pais_vasco,amurrio/alava,amurrio/araba,amurrio/pais_vasco,amurrio/pais_vasco,amurrio/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco
3,01002,01468,amurrio,amurrio,alava,araba,pais_vasco,pais_vasco,amurrio/alava,amurrio/araba,amurrio/pais_vasco,amurrio/pais_vasco,amurrio/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco
4,01002,01470,amurrio,amurrio,alava,araba,pais_vasco,pais_vasco,amurrio/alava,amurrio/araba,amurrio/pais_vasco,amurrio/pais_vasco,amurrio/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco


In [1073]:
maestro_municipios.shape

(14288, 17)

In [1074]:
def union_muni(df_maestro, df_union, muni_maestro, agrup_maestro, muni_union, agrup_union, threshold=85, threshold_mun=90):
    
    count_maestro = df_maestro.shape[0]
    count_union = df_union.shape[0]
    print('num registros maestro: {}'.format(count_maestro))
    print('num registros fichero para unir: {}'.format(count_union))
    
    #fuzzy por prov/ccaa
    maestro_agrup = pd.DataFrame(df_maestro[agrup_maestro].drop_duplicates())
    print('num agrupaciones maestro: {}'.format(maestro_agrup.shape))
    union_agrup = pd.DataFrame(df_union[agrup_union].drop_duplicates())
    print('num agrupaciones union: {}'.format(union_agrup.shape))

    agrup_match = fuzzy_merge(maestro_agrup, union_agrup, agrup_maestro, agrup_union, threshold=threshold)
    print('num agrupaciones tras match: {}'.format(agrup_match.shape))
    num_matches = (agrup_match[agrup_match['matches']!='']['matches'].nunique())
    if (num_matches != agrup_match[agrup_maestro].nunique()):
        print('no se han cruzado todos los valores:')
        print(agrup_match[agrup_match['matches']==''])

        #se genera un nuevo df con el match y el fichero de union
    df_union_agrup_match = pd.merge(df_union, agrup_match, left_on=agrup_union, right_on='matches', how='left')
    if (count_union != df_union_agrup_match.shape[0]):
        print('el fichero de union no dispone de todos los valores para cruzar')    

    df_union_agrup_match.drop(columns=['matches'], inplace=True)
    
    #fuzzy por municipio
    df_maestro['muni_agrup'] = df_maestro[muni_maestro]+"/"+df_maestro[agrup_maestro]
    df_union_agrup_match['muni_agrup'] = df_union_agrup_match[muni_union]+"/"+df_union_agrup_match[agrup_maestro]
   
    muni_match = pd.DataFrame()
    maestro_count = 0
    for agrup in list(maestro_agrup[agrup_maestro]):
        print(agrup)
        maestro_agrup_sel = df_maestro[df_maestro[agrup_maestro] == agrup].copy()
        maestro_count = maestro_agrup_sel.shape[0] + maestro_count
        union_agrup_sel = df_union_agrup_match[df_union_agrup_match[agrup_maestro] == agrup].copy()
        aux = fuzzy_merge(maestro_agrup_sel, union_agrup_sel, 'muni_agrup', 'muni_agrup', threshold=threshold_mun)
        num_matches = (aux[aux['matches']!='']['matches'].nunique())
        if (num_matches != aux[muni_maestro].nunique()):
            print('no se han cruzado todos los valores:')
            #print(aux[aux['matches'].isnull()]) 
        muni_match = pd.concat([muni_match, aux])
        
        #comprobacion registros    
    print('num muni maestro: {}'.format(maestro_count))
    if maestro_count != muni_match.shape[0]:
        print("se han perdido registros")       
        
    muni_match.drop(columns='muni_agrup', inplace=True)
    df_union_agrup_match.drop(columns=[agrup_maestro], inplace=True)

        #se genera un nuevo df con el match y el fichero de union
    df_union_muni_match = pd.merge(muni_match, df_union_agrup_match, left_on='matches', right_on='muni_agrup', how='left')
    if (count_union != df_union_muni_match.shape[0]):
        print('el fichero de union no dispone de todos los valores para cruzar por municipio')        

    #merge maestro con fichero union
    #df_tras_union = pd.merge(df_maestro, df_union_muni_match, left_on=muni_maestro, right_on='matches', how='left')
    
    print('columnas final: {}'.format(df_union_muni_match.columns))
        #borramos columnas que sobran
    if muni_maestro != muni_union:
        df_union_muni_match.drop(columns=muni_union, inplace=True)
    if agrup_maestro != agrup_union:
        df_union_muni_match.drop(columns=agrup_union, inplace=True)
    df_union_muni_match.drop(columns=['matches', 'muni_agrup'], inplace=True)
    print('num registros tras union: {}'.format(df_union_muni_match.shape))
    
    return df_union_muni_match

### Fichero nombres originales

In [1075]:
municipios = pd.read_csv("listado-codigos-postales-con-LatyLon.csv", sep=';', dtype={'codigopostalid': object})
municipios.shape

(14665, 5)

In [1076]:
municipios.head(2)

,provincia,poblacion,codigopostalid,lat,lon
0,Araba/Álava,Alegría-Dulantzi,00240,-2.712437,42.939812
1,Araba/Álava,Alegría-Dulantzi,01193,-2.712437,42.939812


In [1077]:
#municipios = municipios[municipios['provincia_limpio']!='albacete']

In [1078]:
municipios.drop_duplicates(subset=['poblacion','provincia'], inplace=True)
municipios['provincia_limpio'] = municipios['provincia'].apply(doClean)
municipios['municipio_limpio'] = municipios['poblacion'].apply(doClean)
municipios.shape

(8098, 7)

In [1079]:
municipios.drop(columns=['codigopostalid','lat','lon'], inplace=True)

In [1080]:
maestro_nuevo = union_muni(maestro_municipios, municipios, 'cities', 'province', 'municipio_limpio', 'provincia_limpio', threshold=85)

num registros maestro: 14288
num registros fichero para unir: 8098
num agrupaciones maestro: (52, 1)
num agrupaciones union: (52, 1)
num agrupaciones tras match: (52, 2)
araba
no se han cruzado todos los valores:
albacete
alicante
no se han cruzado todos los valores:
murcia
no se han cruzado todos los valores:
almeria
avila
badajoz
illes_balears
no se han cruzado todos los valores:
barcelona
no se han cruzado todos los valores:
tarragona
no se han cruzado todos los valores:
burgos
no se han cruzado todos los valores:
cantabria
no se han cruzado todos los valores:
la_rioja
no se han cruzado todos los valores:
palencia
no se han cruzado todos los valores:
caceres
cadiz
castellon
no se han cruzado todos los valores:
ciudad_real
cordoba
no se han cruzado todos los valores:
granada
no se han cruzado todos los valores:
coruna
cuenca
leon
no se han cruzado todos los valores:
girona
no se han cruzado todos los valores:
guadalajara
madrid
no se han cruzado todos los valores:
gipuzkoa
huelva
hue

In [1067]:
maestro_nuevo.to_csv("maestro_nuevo.csv")

In [1069]:
maestro_nuevo.head()

,ine_code,postal_code,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province_es,cities_es/province,cities_es/ccaa_es,cities_es/ccaa,cities/province_es,province_es/ccaa_es,province_es/ccaa,province/ccaa_es,province/ccaa,provincia,poblacion,muni_agrup
0,1001,01240,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/alava,alegria_dulantzi/araba,alegria_dulantzi/pais_vasco,alegria_dulantzi/pais_vasco,alegria_dulantzi/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco,Araba/Álava,Alegría-Dulantzi,alegria_dulantzi/pais_vasco
1,1001,01193,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/alava,alegria_dulantzi/araba,alegria_dulantzi/pais_vasco,alegria_dulantzi/pais_vasco,alegria_dulantzi/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco,Araba/Álava,Alegría-Dulantzi,alegria_dulantzi/pais_vasco
2,1002,01450,amurrio,amurrio,alava,araba,pais_vasco,pais_vasco,amurrio/alava,amurrio/araba,amurrio/pais_vasco,amurrio/pais_vasco,amurrio/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco,Araba/Álava,Amurrio,amurrio/pais_vasco
3,1002,01468,amurrio,amurrio,alava,araba,pais_vasco,pais_vasco,amurrio/alava,amurrio/araba,amurrio/pais_vasco,amurrio/pais_vasco,amurrio/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco,Araba/Álava,Amurrio,amurrio/pais_vasco
4,1002,01470,amurrio,amurrio,alava,araba,pais_vasco,pais_vasco,amurrio/alava,amurrio/araba,amurrio/pais_vasco,amurrio/pais_vasco,amurrio/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco,Araba/Álava,Amurrio,amurrio/pais_vasco


In [955]:
maestro_nuevo[maestro_nuevo['cities_es']=='benicasim']

,ine_code,postal_code,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province_es,cities_es/province,cities_es/ccaa_es,cities_es/ccaa,cities/province_es,province_es/ccaa_es,province_es/ccaa,province/ccaa_es,province/ccaa,provincia,poblacion
4065,12028,12560,benicasim,benicassim,castellon,castellon,comunidad_valenciana,comunitat_valenciana,benicasim/castellon,benicasim/castellon,benicasim/comunidad_valenciana,benicasim/comunidad_valenciana,benicassim/castellon,castellon/comunidad_valenciana,castellon/comunidad_valenciana,castellon/comunidad_valenciana,castellon/comunidad_valenciana,Castellón/Castelló,Benicasim/Benicàssim
4079,12040,12560,benicasim,benicassim,castellon,castellon,comunidad_valenciana,comunitat_valenciana,benicasim/castellon,benicasim/castellon,benicasim/comunidad_valenciana,benicasim/comunidad_valenciana,benicassim/castellon,castellon/comunidad_valenciana,castellon/comunidad_valenciana,castellon/comunidad_valenciana,castellon/comunidad_valenciana,Castellón/Castelló,Benicasim/Benicàssim


In [20]:
maestro_nuevo[maestro_nuevo['province_es']=='castellon'].head(2)

,ine_code,postal_code,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province_es,cities_es/province,cities_es/ccaa_es,cities_es/ccaa,cities/province_es,province_es/ccaa_es,province_es/ccaa,province/ccaa_es,province/ccaa
2875,12001,12132,atzeneta_del_maestrat,atzeneta_del_maestrat,castellon,castello,comunidad_valenciana,comunitat_valenciana,atzeneta_del_maestrat/castellon,atzeneta_del_maestrat/castello,atzeneta_del_maestrat/comunidad_valenciana,atzeneta_del_maestrat/comunidad_valenciana,atzeneta_del_maestrat/castellon,castellon/comunidad_valenciana,castellon/comunidad_valenciana,castellon/comunidad_valenciana,castellon/comunidad_valenciana
2876,12001,12133,atzeneta_del_maestrat,atzeneta_del_maestrat,castellon,castello,comunidad_valenciana,comunitat_valenciana,atzeneta_del_maestrat/castellon,atzeneta_del_maestrat/castello,atzeneta_del_maestrat/comunidad_valenciana,atzeneta_del_maestrat/comunidad_valenciana,atzeneta_del_maestrat/castellon,castellon/comunidad_valenciana,castellon/comunidad_valenciana,castellon/comunidad_valenciana,castellon/comunidad_valenciana


## Ficheros especificos

##### Bibliotecas

In [956]:
bibliotecas = pd.read_csv("sociedad_cultura_bibliotecas_2017_limipio.csv", index_col=0)

In [957]:
bibliotecas.shape

(3323, 4)

In [958]:
bibliotecas.head(2)

,municipio_limpio,ccaa,provincia,total_bibliotecas
0,alaior,illes_balears,illes_balears,1
1,alaro,illes_balears,illes_balears,1


In [959]:
bibliotecas[bibliotecas['municipio_limpio']=='soneja']

,municipio_limpio,ccaa,provincia,total_bibliotecas


In [960]:
bibliotecas[bibliotecas['municipio_limpio']=='palma']
# idioma municipio = es

,municipio_limpio,ccaa,provincia,total_bibliotecas
38,palma,illes_balears,illes_balears,22


In [961]:
bibliotecas['ccaa_limpio'] = bibliotecas['ccaa']

In [962]:
bibliotecas.drop(columns=['provincia','ccaa'], inplace=True)

In [963]:
maestro_nuevo2 = union_muni(maestro_nuevo, bibliotecas, 'cities', 'ccaa', 'municipio_limpio', 'ccaa_limpio', threshold=85)

num registros maestro: 14288
num registros fichero para unir: 3323
num agrupaciones maestro: (19, 1)
num agrupaciones union: (19, 1)
num agrupaciones tras match: (19, 2)
pais_vasco
no se han cruzado todos los valores:
castilla_la_mancha
no se han cruzado todos los valores:
comunitat_valenciana
no se han cruzado todos los valores:
region_de_murcia
no se han cruzado todos los valores:
andalucia
no se han cruzado todos los valores:
castilla_y_leon
no se han cruzado todos los valores:
extremadura
no se han cruzado todos los valores:
illes_balears
no se han cruzado todos los valores:
cataluna
no se han cruzado todos los valores:
cantabria
no se han cruzado todos los valores:
la_rioja
no se han cruzado todos los valores:
galicia
no se han cruzado todos los valores:
comunidad_de_madrid
no se han cruzado todos los valores:
aragon
no se han cruzado todos los valores:
comunidad_foral_de_navarra
no se han cruzado todos los valores:
principado_de_asturias
no se han cruzado todos los valores:
canar

In [965]:
maestro_nuevo2[maestro_nuevo2['cities']=='soneja']

,ine_code,postal_code,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province_es,cities_es/province,cities_es/ccaa_es,cities_es/ccaa,cities/province_es,province_es/ccaa_es,province_es/ccaa,province/ccaa_es,province/ccaa,provincia,poblacion,total_bibliotecas
1995,12106,12480,soneja,soneja,castellon,castellon,comunidad_valenciana,comunitat_valenciana,soneja/castellon,soneja/castellon,soneja/comunidad_valenciana,soneja/comunidad_valenciana,soneja/castellon,castellon/comunidad_valenciana,castellon/comunidad_valenciana,castellon/comunidad_valenciana,castellon/comunidad_valenciana,Castellón/Castelló,Soneja,NaN


In [966]:
maestro_nuevo2[~maestro_nuevo2['total_bibliotecas'].isnull()].shape

(8000, 20)

In [967]:
maestro_nuevo2.shape

(14288, 20)

##### Hospitales

In [34]:
hospitales = pd.read_csv("sociedad_sanidad_hospitales_2019_limpio.csv", index_col=0, dtype={'cp': object})

In [35]:
hospitales.shape

(613, 2)

In [36]:
hospitales.head(2)

,cp,uds_hospiatales
0,01002,1
1,01006,1


In [37]:
hospitales[hospitales['cp']=='46010']

,cp,uds_hospiatales
549,46010,2


In [38]:
union = pd.merge(union, hospitales, left_on='postal_code', right_on='cp', how='left')

In [39]:
union[~union['uds_hospiatales'].isnull()].shape

(650, 20)

In [40]:
union.head(2)

,ine_code,postal_code,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province_es,cities_es/province,cities_es/ccaa_es,cities_es/ccaa,cities/province_es,province_es/ccaa_es,province_es/ccaa,province/ccaa_es,province/ccaa,total_bibliotecas,cp,uds_hospiatales
0,1001,01240,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/alava,alegria_dulantzi/araba,alegria_dulantzi/pais_vasco,alegria_dulantzi/pais_vasco,alegria_dulantzi/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco,1.0,NaN,NaN
1,1001,01193,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/alava,alegria_dulantzi/araba,alegria_dulantzi/pais_vasco,alegria_dulantzi/pais_vasco,alegria_dulantzi/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco,1.0,NaN,NaN


In [41]:
union = union.drop(columns=['cp'])

In [42]:
union.shape

(14729, 19)

##### Centros salud

In [43]:
centros_salud = pd.read_csv("sociedad_sanidad_centros_salud_2019_limpio.csv", index_col=0, dtype={'cp': object})

In [44]:
centros_salud.shape

(8316, 2)

In [45]:
centros_salud.head(2)

,cp,num_centros_salud
0,01001,1
1,01002,2


In [46]:
centros_salud[centros_salud['cp']=='46010']

,cp,num_centros_salud
7249,46010,1


In [47]:
union = pd.merge(union, centros_salud, left_on='postal_code', right_on='cp', how='left')

In [48]:
union[~union['num_centros_salud'].isnull()].shape

(11332, 21)

In [49]:
union.head(2)

,ine_code,postal_code,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province_es,cities_es/province,...,cities_es/ccaa,cities/province_es,province_es/ccaa_es,province_es/ccaa,province/ccaa_es,province/ccaa,total_bibliotecas,uds_hospiatales,cp,num_centros_salud
0,1001,01240,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/alava,alegria_dulantzi/araba,...,alegria_dulantzi/pais_vasco,alegria_dulantzi/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco,1.0,NaN,01240,1.0
1,1001,01193,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/alava,alegria_dulantzi/araba,...,alegria_dulantzi/pais_vasco,alegria_dulantzi/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco,1.0,NaN,NaN,NaN


In [50]:
union = union.drop(columns=['cp'])

In [51]:
union.shape

(14729, 20)

##### Colegios

In [1033]:
colegios = pd.read_csv("sociedad_educacion_colegios_2020_limpio.csv", index_col=0, dtype={'codigo_postal': object})

In [1041]:
colegios.shape

(6331, 4)

In [1035]:
colegios.head(2)

,codigo_postal,municipio_limpio,provincia,num_colegios_privados,num_colegios_publicos
0,00000,a_coruna,a_coruna,6.0,NaN
1,00000,a_pontenova,lugo,NaN,1.0


In [1036]:
colegios[colegios['codigo_postal']=='01450']
#codigos postales erroneos, se une por municipio/provincia y cambiando de idioma

,codigo_postal,municipio_limpio,provincia,num_colegios_privados,num_colegios_publicos


In [1037]:
colegios[colegios['municipio_limpio']=='alegria_dulantzi']
# idioma municipio = es

,codigo_postal,municipio_limpio,provincia,num_colegios_privados,num_colegios_publicos
81,01240,alegria_dulantzi,araba,NaN,3.0


In [1038]:
colegios['provincia_limpio'] = colegios['provincia']
colegios.drop(columns=['provincia','codigo_postal'], inplace=True)

In [1040]:
colegios = colegios.groupby(by=['municipio_limpio','provincia_limpio'], as_index=False).sum()

In [1042]:
maestro_nuevo3 = union_muni(maestro_nuevo2, colegios, 'cities_es', 'province', 'municipio_limpio', 'provincia_limpio', threshold=85, threshold_mun=90)

num registros maestro: 14288
num registros fichero para unir: 6331
num agrupaciones maestro: (52, 1)
num agrupaciones union: (52, 1)
num agrupaciones tras match: (52, 2)
araba
no se han cruzado todos los valores:
gipuzkoa
no se han cruzado todos los valores:
bizkaia
no se han cruzado todos los valores:
albacete
no se han cruzado todos los valores:
ciudad_real
no se han cruzado todos los valores:
cuenca
no se han cruzado todos los valores:
guadalajara
no se han cruzado todos los valores:
toledo
no se han cruzado todos los valores:
alicante
no se han cruzado todos los valores:
castellon
no se han cruzado todos los valores:
valencia
no se han cruzado todos los valores:
murcia
no se han cruzado todos los valores:
almeria
no se han cruzado todos los valores:
cadiz
no se han cruzado todos los valores:
cordoba
no se han cruzado todos los valores:
granada
no se han cruzado todos los valores:
huelva
no se han cruzado todos los valores:
jaen
no se han cruzado todos los valores:
malaga
no se han 

In [1050]:
maestro_nuevo3[maestro_nuevo3['num_colegios_publicos']>0.0].shape

(11363, 22)

In [1051]:
maestro_nuevo3.columns

Index(['ine_code', 'postal_code', 'cities_es', 'cities', 'province_es',
       'province', 'ccaa_es', 'ccaa', 'cities_es/province_es',
       'cities_es/province', 'cities_es/ccaa_es', 'cities_es/ccaa',
       'cities/province_es', 'province_es/ccaa_es', 'province_es/ccaa',
       'province/ccaa_es', 'province/ccaa', 'provincia', 'poblacion',
       'total_bibliotecas', 'num_colegios_privados', 'num_colegios_publicos'],
      dtype='object')

In [1052]:
maestro_nuevo3.shape

(14288, 22)

##### Guardia Civil

In [1053]:
guardia_civil = pd.read_csv("sociedad_seguridad_guardia_civil_2019_limpio.csv", index_col=0, dtype={'cp': object})

In [1064]:
guardia_civil.shape

(2139, 3)

In [1066]:
guardia_civil.head(20)
#codigo postal no valido

,municipio_limpio,provincia_limpio,uds_guardia_civil
0,a_ca_iza,pontevedra,1
1,a_coruna,a_coruna,2
2,a_estrada,pontevedra,1
3,a_feira_do_monte,lugo,1
4,a_guarda,pontevedra,1
5,a_gudi_a,orense,1
6,a_mezquita,orense,1
7,a_over_de_tajo,toledo,1
8,a_pontenova,lugo,1
9,a_rua,orense,1


In [1056]:
guardia_civil[guardia_civil['municipio_limpio']=='benicasim']
# idioma municipio = es

,cp,municipio_limpio,provincia,uds_guardia_civil
153,12560,benicasim,castellon,1


In [1059]:
guardia_civil['provincia_limpio'] = guardia_civil['provincia']
guardia_civil.drop(columns=['provincia','cp'], inplace=True)

In [1060]:
guardia_civil = guardia_civil.groupby(by=['municipio_limpio','provincia_limpio'], as_index=False).sum()

In [1063]:
maestro_nuevo4 = union_muni(maestro_nuevo3, guardia_civil, 'cities_es', 'province', 'municipio_limpio', 'provincia_limpio', threshold=85, threshold_mun=90)

num registros maestro: 14288
num registros fichero para unir: 2139
num agrupaciones maestro: (52, 1)
num agrupaciones union: (52, 1)
num agrupaciones tras match: (52, 2)
no se han cruzado todos los valores:
    province matches
0      araba        
266  bizkaia        
araba


RuntimeError: generator raised StopIteration

In [1057]:
maestro_municipios[maestro_municipios['cities_es']=='benicasim']

,ine_code,postal_code,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province_es,cities_es/province,cities_es/ccaa_es,cities_es/ccaa,cities/province_es,province_es/ccaa_es,province_es/ccaa,province/ccaa_es,province/ccaa,muni_agrup
2905,12028,12560,benicasim,benicassim,castellon,castellon,comunidad_valenciana,comunitat_valenciana,benicasim/castellon,benicasim/castellon,benicasim/comunidad_valenciana,benicasim/comunidad_valenciana,benicassim/castellon,castellon/comunidad_valenciana,castellon/comunidad_valenciana,castellon/comunidad_valenciana,castellon/comunidad_valenciana,benicassim/castellon
2919,12040,12560,benicasim,benicassim,castellon,castellon,comunidad_valenciana,comunitat_valenciana,benicasim/castellon,benicasim/castellon,benicasim/comunidad_valenciana,benicasim/comunidad_valenciana,benicassim/castellon,castellon/comunidad_valenciana,castellon/comunidad_valenciana,castellon/comunidad_valenciana,castellon/comunidad_valenciana,benicassim/castellon


In [74]:
maestro_municipios['province_es'].drop_duplicates().sort_values()

0                         alava
132                    albacete
305                    alicante
549                     almeria
9209                   asturias
768                       avila
1063                    badajoz
1463                  barcelona
1911                     burgos
2455                    caceres
2759                      cadiz
1918                  cantabria
2875                  castellon
14718                     ceuta
3044                ciudad_real
3070                    cordoba
3379                     coruna
3801                     cuenca
4087                     girona
3311                    granada
4663                guadalajara
5051                  guipuzkoa
5183                     huelva
5214                     huesca
1283             islas_baleares
5738                       jaen
2170                   la_rioja
9962                 las_palmas
3985                       leon
6509                     lleida
7124                       lugo
4739    

In [75]:
guardia_civil['provincia'].replace("illes_balears", "islas_baleares", inplace=True )

In [76]:
# listado de provincias y ccaa
guardia_civil['provincia'].drop_duplicates().sort_values()

284                   a_coruna
1                        alava
510                   albacete
1032                  alicante
1492                   almeria
1182                  asturias
1944                     avila
2024                   badajoz
2166                 barcelona
2200                    burgos
0                      caceres
75                       cadiz
1449                 cantabria
128                  castellon
1994                     ceuta
164                ciudad_real
231                    cordoba
341                     cuenca
509                   gipuzkoa
388                     girona
400                    granada
477                guadalajara
520                     huelva
585                     huesca
2117            islas_baleares
639                       jaen
777                   la_rioja
1295                las_palmas
711                       leon
759                     lleida
819                       lugo
871                     madrid
973     

In [77]:
guardia_civil['cities_es/province_es'] = guardia_civil['municipio_limpio']+"/"+guardia_civil['provincia']

In [78]:
guardia_civil = guardia_civil.groupby(by='cities_es/province_es', as_index=False).agg({'uds_guardia_civil':'sum'})

In [79]:
union = pd.merge(union, guardia_civil, left_on='cities_es/province_es', right_on='cities_es/province_es', how='left')

In [80]:
union[~union['uds_guardia_civil'].isnull()].shape

(4763, 23)

In [81]:
union.head(2)

,ine_code,postal_code,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province_es,cities_es/province,...,province_es/ccaa_es,province_es/ccaa,province/ccaa_es,province/ccaa,total_bibliotecas,uds_hospiatales,num_centros_salud,num_colegios_privados,num_colegios_publicos,uds_guardia_civil
0,1001,01240,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/alava,alegria_dulantzi/araba,...,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco,1.0,NaN,1.0,0.0,3.0,NaN
1,1001,01193,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/alava,alegria_dulantzi/araba,...,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco,1.0,NaN,NaN,0.0,3.0,NaN


In [82]:
union.shape

(14729, 23)

##### Universidades

In [83]:
universidad = pd.read_csv('sociedad_educacion_universidad_2018_limpio.csv', index_col=0, dtype={'cp': object})

In [84]:
universidad.shape

(165, 6)

In [85]:
universidad.head(2)

,municipio_limpio,ccaa,num_universidades_privadas,num_universidades_publicas,num_tit_uni_privadas,num_tit_uni_publicas
0,albacete,castilla_la_mancha,NaN,1.0,NaN,52.0
1,alcala_de_henares,madrid,1.0,1.0,5.0,114.0


In [86]:
universidad[universidad['municipio_limpio']=='alcoy']

,municipio_limpio,ccaa,num_universidades_privadas,num_universidades_publicas,num_tit_uni_privadas,num_tit_uni_publicas
5,alcoy,comunitat_valenciana,NaN,1.0,NaN,12.0


In [87]:
maestro_municipios[maestro_municipios['cities']=='alcoy']

,ine_code,postal_code,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province_es,cities_es/province,cities_es/ccaa_es,cities_es/ccaa,cities/province_es,province_es/ccaa_es,province_es/ccaa,province/ccaa_es,province/ccaa
315,3009,03804,alcoi,alcoy,alicante,alacant,comunidad_valenciana,comunitat_valenciana,alcoi/alicante,alcoi/alacant,alcoi/comunidad_valenciana,alcoi/comunidad_valenciana,alcoy/alicante,alicante/comunidad_valenciana,alicante/comunidad_valenciana,alicante/comunidad_valenciana,alicante/comunidad_valenciana
316,3009,03803,alcoi,alcoy,alicante,alacant,comunidad_valenciana,comunitat_valenciana,alcoi/alicante,alcoi/alacant,alcoi/comunidad_valenciana,alcoi/comunidad_valenciana,alcoy/alicante,alicante/comunidad_valenciana,alicante/comunidad_valenciana,alicante/comunidad_valenciana,alicante/comunidad_valenciana
317,3009,03802,alcoi,alcoy,alicante,alacant,comunidad_valenciana,comunitat_valenciana,alcoi/alicante,alcoi/alacant,alcoi/comunidad_valenciana,alcoi/comunidad_valenciana,alcoy/alicante,alicante/comunidad_valenciana,alicante/comunidad_valenciana,alicante/comunidad_valenciana,alicante/comunidad_valenciana
318,3009,03800,alcoi,alcoy,alicante,alacant,comunidad_valenciana,comunitat_valenciana,alcoi/alicante,alcoi/alacant,alcoi/comunidad_valenciana,alcoi/comunidad_valenciana,alcoy/alicante,alicante/comunidad_valenciana,alicante/comunidad_valenciana,alicante/comunidad_valenciana,alicante/comunidad_valenciana
319,3009,03801,alcoi,alcoy,alicante,alacant,comunidad_valenciana,comunitat_valenciana,alcoi/alicante,alcoi/alacant,alcoi/comunidad_valenciana,alcoi/comunidad_valenciana,alcoy/alicante,alicante/comunidad_valenciana,alicante/comunidad_valenciana,alicante/comunidad_valenciana,alicante/comunidad_valenciana


In [88]:
universidad['ccaa'].drop_duplicates().sort_values()

7                andalucia
11                  aragon
60                asturias
50                 balears
18                canarias
42               cantabria
0       castilla_la_mancha
19         castilla_y_leon
21                cataluna
2     comunitat_valenciana
9              extremadura
45                 galicia
1                   madrid
32                  murcia
22                 navarra
16              pais_vasco
78                   rioja
Name: ccaa, dtype: object

In [89]:
universidad['ccaa'].replace("balears","illes_balears", inplace=True, regex=True)
universidad['ccaa'].replace("madrid","comunidad_de_madrid", inplace=True, regex=True)
universidad['ccaa'].replace("navarra","comunidad_foral_de_navarra", inplace=True, regex=True)
universidad['ccaa'].replace("asturias","principado_de_asturias", inplace=True, regex=True)
universidad['ccaa'].replace("rioja","la_rioja", inplace=True, regex=True)
universidad['ccaa'].replace("murcia","region_de_murcia", inplace=True, regex=True)

In [90]:
maestro_municipios['ccaa'].drop_duplicates().sort_values()

549                       andalucia
5214                         aragon
9962                       canarias
1918                      cantabria
132              castilla_la_mancha
768                 castilla_y_leon
1463                       cataluna
14718                         ceuta
4739            comunidad_de_madrid
8408     comunidad_foral_de_navarra
305            comunitat_valenciana
1063                    extremadura
3379                        galicia
1283                  illes_balears
2170                       la_rioja
14723                       melilla
0                        pais_vasco
9209         principado_de_asturias
496                region_de_murcia
134                             NaN
Name: ccaa, dtype: object

In [91]:
universidad['cities_es/ccaa'] = universidad['municipio_limpio']+"/"+universidad['ccaa']

In [92]:
universidad.head()

,municipio_limpio,ccaa,num_universidades_privadas,num_universidades_publicas,num_tit_uni_privadas,num_tit_uni_publicas,cities_es/ccaa
0,albacete,castilla_la_mancha,NaN,1.0,NaN,52.0,albacete/castilla_la_mancha
1,alcala_de_henares,comunidad_de_madrid,1.0,1.0,5.0,114.0,alcala_de_henares/comunidad_de_madrid
2,alcobendas,comunitat_valenciana,1.0,NaN,6.0,NaN,alcobendas/comunitat_valenciana
3,alcobendas,comunidad_de_madrid,3.0,NaN,26.0,NaN,alcobendas/comunidad_de_madrid
4,alcorcon,comunidad_de_madrid,NaN,1.0,NaN,21.0,alcorcon/comunidad_de_madrid


In [93]:
universidad.drop(columns= ['municipio_limpio','ccaa'], inplace=True)

In [94]:
union = pd.merge(union, universidad, left_on='cities_es/ccaa_es', right_on='cities_es/ccaa', how='left')

In [95]:
union[~union['num_universidades_publicas'].isnull()].shape

(1204, 28)

In [96]:
union.head(2)

,ine_code,postal_code,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province_es,cities_es/province,...,uds_hospiatales,num_centros_salud,num_colegios_privados,num_colegios_publicos,uds_guardia_civil,num_universidades_privadas,num_universidades_publicas,num_tit_uni_privadas,num_tit_uni_publicas,cities_es/ccaa_y
0,1001,01240,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/alava,alegria_dulantzi/araba,...,NaN,1.0,0.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1001,01193,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/alava,alegria_dulantzi/araba,...,NaN,NaN,0.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN


In [97]:
union.shape

(14729, 28)

##### Establecimientos

In [98]:
establecimientos = pd.read_csv("economia_turismo_establecimientos_20XX_limpio.csv", index_col=0, dtype={'codigo_postal': object})

In [99]:
establecimientos.shape

(2063, 35)

In [100]:
establecimientos.head(2)

,codigo_postal,municipio_limpio,ccaa,agencias_de_viajes,albergues,alojamientos_rurales,artesanos,atraque_de_cruceros_y_ferries,bares_y_cafeterías,bodegas,...,otros_servicios,palacios_de_congresos,playas,puertos_deportivos,restaurantes_y_empresas_turísticas_de_catering,seguridad_ciudadana,servicios_de_limpieza,servicios_turísticos_de_salud,taxis,turismo_industrial
0,00000,aldea_de_cuenca_hinojares,andalucia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00000,arroyo_frio_la_iruela,andalucia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN


In [101]:
# mas de un codigo postal por municipio, se producen duplicados
establecimientos[establecimientos['codigo_postal']=='01118']

,codigo_postal,municipio_limpio,ccaa,agencias_de_viajes,albergues,alojamientos_rurales,artesanos,atraque_de_cruceros_y_ferries,bares_y_cafeterías,bodegas,...,otros_servicios,palacios_de_congresos,playas,puertos_deportivos,restaurantes_y_empresas_turísticas_de_catering,seguridad_ciudadana,servicios_de_limpieza,servicios_turísticos_de_salud,taxis,turismo_industrial
18,01118,bernedo,euskadi_pais_vasco,NaN,1.0,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,01118,lagran,euskadi_pais_vasco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,01118,urturi,euskadi_pais_vasco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
establecimientos[establecimientos['municipio_limpio']=='alcoy']
# municipio en 2 idiomas, se cruza por cp

,codigo_postal,municipio_limpio,ccaa,agencias_de_viajes,albergues,alojamientos_rurales,artesanos,atraque_de_cruceros_y_ferries,bares_y_cafeterías,bodegas,...,otros_servicios,palacios_de_congresos,playas,puertos_deportivos,restaurantes_y_empresas_turísticas_de_catering,seguridad_ciudadana,servicios_de_limpieza,servicios_turísticos_de_salud,taxis,turismo_industrial
164,03800,alcoy,comunitat_valenciana,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
166,03801,alcoy,comunitat_valenciana,NaN,NaN,NaN,NaN,NaN,4.0,NaN,...,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN
168,03802,alcoy,comunitat_valenciana,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN
169,03803,alcoy,comunitat_valenciana,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
171,03804,alcoy,comunitat_valenciana,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN


In [103]:
establecimientos[establecimientos['codigo_postal'].duplicated()]

,codigo_postal,municipio_limpio,ccaa,agencias_de_viajes,albergues,alojamientos_rurales,artesanos,atraque_de_cruceros_y_ferries,bares_y_cafeterías,bodegas,...,otros_servicios,palacios_de_congresos,playas,puertos_deportivos,restaurantes_y_empresas_turísticas_de_catering,seguridad_ciudadana,servicios_de_limpieza,servicios_turísticos_de_salud,taxis,turismo_industrial
1,00000,arroyo_frio_la_iruela,andalucia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2,00000,cazorla,andalucia,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00000,la_iruela,andalucia,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00000,pozo_alcon,andalucia,NaN,1.0,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,01117,san_vicente_arana,euskadi_pais_vasco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2049,50513,los_fayos,aragon,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2050,50513,santa_cruz_de_moncayo,aragon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2051,50513,santa_cruz_del_moncayo,aragon,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2052,50513,torrellas,aragon,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [104]:
# listado de provincias y ccaa
establecimientos[establecimientos['codigo_postal']=='50513']

,codigo_postal,municipio_limpio,ccaa,agencias_de_viajes,albergues,alojamientos_rurales,artesanos,atraque_de_cruceros_y_ferries,bares_y_cafeterías,bodegas,...,otros_servicios,palacios_de_congresos,playas,puertos_deportivos,restaurantes_y_empresas_turísticas_de_catering,seguridad_ciudadana,servicios_de_limpieza,servicios_turísticos_de_salud,taxis,turismo_industrial
2047,50513,anon_de_moncayo,aragon,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2048,50513,grisel,aragon,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2049,50513,los_fayos,aragon,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2050,50513,santa_cruz_de_moncayo,aragon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2051,50513,santa_cruz_del_moncayo,aragon,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2052,50513,torrellas,aragon,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [105]:
establecimientos.drop(columns=['municipio_limpio','ccaa'], inplace=True)

In [106]:
establecimientos = establecimientos.groupby(['codigo_postal'],as_index=False).agg(lambda x : x.sum() if x.dtype=='float64' else x.head(1))

In [107]:
union = pd.merge(union, establecimientos, left_on='postal_code', right_on='codigo_postal', how='left')

In [108]:
union[~union['agencias_de_viajes'].isnull()].shape

(1826, 61)

In [109]:
union.columns

Index(['ine_code', 'postal_code', 'cities_es', 'cities', 'province_es',
       'province', 'ccaa_es', 'ccaa', 'cities_es/province_es',
       'cities_es/province', 'cities_es/ccaa_es', 'cities_es/ccaa_x',
       'cities/province_es', 'province_es/ccaa_es', 'province_es/ccaa',
       'province/ccaa_es', 'province/ccaa', 'total_bibliotecas',
       'uds_hospiatales', 'num_centros_salud', 'num_colegios_privados',
       'num_colegios_publicos', 'uds_guardia_civil',
       'num_universidades_privadas', 'num_universidades_publicas',
       'num_tit_uni_privadas', 'num_tit_uni_publicas', 'cities_es/ccaa_y',
       'codigo_postal', 'agencias_de_viajes', 'albergues',
       'alojamientos_rurales', 'artesanos', 'atraque_de_cruceros_y_ferries',
       'bares_y_cafeterías', 'bodegas', 'campings',
       'campos_de_golf_y_campos_de_pitch&putt', 'comercios',
       'convention_bureaux',
       'empresas_de_alquiler_de_vehículos_y_embarcaciones',
       'empresas_de_transporte_turístico', 'empresas_

In [110]:
union.shape

(14729, 61)

##### Usos suelo

In [111]:
usos_suelo = pd.read_csv("sociedad_usos_suelo_2019_limpio.csv", index_col=0)

In [112]:
usos_suelo.shape

(7610, 12)

In [113]:
usos_suelo.head(2)

,municipio_limpio,provincia,superficie_urbana,uds_almacen,uds_comercial,uds_cultural,uds_hosteleria,uds_industrial,uds_deportivo,uds_oficinas,uds_religioso,uds_espectaculos
0,abengibre,albacete,"302,079",13,3,2,1,169,1,2,1,0
1,alatoz,albacete,"339,890",41,2,1,1,140,4,2,2,0


In [114]:
usos_suelo['provincia'].drop_duplicates()
# no esta el pais vasco - se deescarta

0                     albacete
87                    alicante
228                    almeria
331                      avila
579                    badajoz
744                    balears
811                  barcelona
1122                    burgos
1493                   caceres
1716                     cadiz
1761                 castellon
1896               ciudad_real
1998                   cordoba
2075                    coruna
2169                    cuenca
2407                    girona
2628                   granada
2802               guadalajara
3090                    huelva
3170                    huesca
3372                      jaen
3469                      leon
3680                    lleida
3911                     rioja
4085                      lugo
4152                    madrid
4331                    malaga
4434                    murcia
4479                   ourense
4571                  asturias
4649                  palencia
4840                    palmas
4874    

In [115]:
union.shape

(14729, 61)

In [116]:
union.head(3)

,ine_code,postal_code,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province_es,cities_es/province,...,otros_servicios,palacios_de_congresos,playas,puertos_deportivos,restaurantes_y_empresas_turísticas_de_catering,seguridad_ciudadana,servicios_de_limpieza,servicios_turísticos_de_salud,taxis,turismo_industrial
0,1001,01240,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/alava,alegria_dulantzi/araba,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1001,01193,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/alava,alegria_dulantzi/araba,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1002,01450,amurrio,amurrio,alava,araba,pais_vasco,pais_vasco,amurrio/alava,amurrio/araba,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [117]:
union.to_csv("union_patri.csv")